In [8]:
import os
import sys
import socket
import pickle
import importlib
import itertools
import functools
import collections
import logging
logging.basicConfig(level=logging.INFO)

import tqdm

import scipy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns

import xgboost as xgb

from statsmodels.stats.multitest import multipletests

import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics as metrics
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate, train_test_split, KFold, GridSearchCV

# Figure out local paths
SRC_DIR = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 'rnagps')
assert os.path.isdir(SRC_DIR)
sys.path.append(SRC_DIR)
import data_loader
import model_utils
import plotting_utils
import utils
import seq

PLOT_DIR = os.path.join(os.path.dirname(SRC_DIR), "plots")
assert os.path.isdir(PLOT_DIR)
plt.style.use(['default', os.path.join(SRC_DIR, 'seaborn-paper-custom.mplstyle')])

MODEL_DIR = os.path.join(os.path.dirname(SRC_DIR), "models")
assert os.path.isdir(MODEL_DIR)

np.random.seed(27492)
SEED = 27492

print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.21.3.


In [2]:
importlib.reload(data_loader)
K_FOLD = 5
nuc_localizations = ["Lma", "Nik", "Nls", "NucPore"]
nuc_train_dataset = data_loader.LocalizationClassificationKmers("train", k_fold=K_FOLD, kmer_sizes=[3, 4, 5], localizations=nuc_localizations, addtl_negatives=False)
nuc_valid_dataset = data_loader.LocalizationClassificationKmers("valid", k_fold=K_FOLD, kmer_sizes=[3, 4, 5], localizations=nuc_localizations, addtl_negatives=False)
nuc_test_dataset = data_loader.LocalizationClassificationKmers("test", k_fold=K_FOLD, kmer_sizes=[3, 4, 5], localizations=nuc_localizations, addtl_negatives=False)
nuc_train_data = data_loader.load_data_as_np(nuc_train_dataset)
nuc_valid_data = data_loader.load_data_as_np(nuc_valid_dataset)
nuc_test_data = data_loader.load_data_as_np(nuc_test_dataset)

INFO:root:Read deseq table containing results for localizations: Erm Kdel Lma Mito Nes Nik Nls NucPore Omm
INFO:root:Retaining data only for Lma Nik Nls NucPore
INFO:root:Retaining 2179/20510 genes as significant
INFO:root:Lma - 613/1743 = 0.3516924842226047 positive
INFO:root:Nik - 1122/1743 = 0.6437177280550774 positive
INFO:root:Nls - 657/1743 = 0.3769363166953528 positive
INFO:root:NucPore - 126/1743 = 0.07228915662650602 positive
INFO:root:1459	0.8370625358577166	protein_coding
INFO:root:75	0.043029259896729774	antisense_RNA
INFO:root:70	0.040160642570281124	lincRNA
INFO:root:37	0.021227768215720023	processed_pseudogene
INFO:root:35	0.020080321285140562	transcribed_unprocessed_pseudogene
INFO:root:18	0.010327022375215147	processed_transcript
INFO:root:7	0.004016064257028112	sense_overlapping
INFO:root:7	0.004016064257028112	unprocessed_pseudogene
INFO:root:7	0.004016064257028112	rRNA
INFO:root:6	0.0034423407917383822	misc_RNA
INFO:root:6	0.0034423407917383822	transcribed_processed

In [3]:
importlib.reload(data_loader)
K_FOLD = 5
cyto_localizations = ["Erm", "Mito", "Nes", "Omm"]
cyto_train_dataset = data_loader.LocalizationClassificationKmers("train", k_fold=K_FOLD, kmer_sizes=[3, 4, 5], localizations=cyto_localizations, addtl_negatives=False)
cyto_valid_dataset = data_loader.LocalizationClassificationKmers("valid", k_fold=K_FOLD, kmer_sizes=[3, 4, 5], localizations=cyto_localizations, addtl_negatives=False)
cyto_test_dataset = data_loader.LocalizationClassificationKmers("test", k_fold=K_FOLD, kmer_sizes=[3, 4, 5], localizations=cyto_localizations, addtl_negatives=False)
cyto_train_data = data_loader.load_data_as_np(cyto_train_dataset)
cyto_valid_data = data_loader.load_data_as_np(cyto_valid_dataset)
cyto_test_data = data_loader.load_data_as_np(cyto_test_dataset)

INFO:root:Read deseq table containing results for localizations: Erm Kdel Lma Mito Nes Nik Nls NucPore Omm
INFO:root:Retaining data only for Erm Mito Nes Omm
INFO:root:Retaining 1860/19511 genes as significant
INFO:root:Erm - 976/1488 = 0.6559139784946236 positive
INFO:root:Mito - 230/1488 = 0.15456989247311828 positive
INFO:root:Nes - 178/1488 = 0.1196236559139785 positive
INFO:root:Omm - 595/1488 = 0.39986559139784944 positive
INFO:root:1430	0.9610215053763441	protein_coding
INFO:root:12	0.008064516129032258	Mt_tRNA
INFO:root:12	0.008064516129032258	antisense_RNA
INFO:root:9	0.006048387096774193	processed_pseudogene
INFO:root:7	0.004704301075268817	transcribed_unprocessed_pseudogene
INFO:root:6	0.004032258064516129	unprocessed_pseudogene
INFO:root:4	0.002688172043010753	lincRNA
INFO:root:3	0.0020161290322580645	transcribed_processed_pseudogene
INFO:root:2	0.0013440860215053765	Mt_rRNA
INFO:root:2	0.0013440860215053765	processed_transcript
INFO:root:1	0.0006720430107526882	transcribed

In [4]:
# Params taken from overall optimal params, not fine tuned here yet
rf_optimal_params = (300, 'entropy', 100, 0.001, 'sqrt', None)
rf_optimal_param_dict = {
    "n_estimators": rf_optimal_params[0],
    "criterion": rf_optimal_params[1],
    "max_depth": rf_optimal_params[2],
    "min_samples_leaf": rf_optimal_params[3],
    "max_features": rf_optimal_params[4],
    "class_weight": rf_optimal_params[5],
    "n_jobs": 32,
    "random_state": 39282,
}

In [6]:
nuc_rf_model, nuc_rf_per_class_perf, nuc_rf_overall_perf = model_utils.multi_train(
    RandomForestClassifier(**rf_optimal_param_dict),
    nuc_train_data,
    nuc_valid_data,
    verbose=False,
)
nuc_rf_overall_perf.auroc, nuc_rf_overall_perf.auprc

(0.7299855859239881, 0.5710418328744873)

In [7]:
cyto_rf_model, cyto_rf_per_class_perf, cyto_rf_overall_perf = model_utils.multi_train(
    RandomForestClassifier(**rf_optimal_param_dict),
    cyto_train_data,
    cyto_valid_data,
    verbose=False,
)
cyto_rf_overall_perf.auroc, cyto_rf_overall_perf.auprc

(0.7725705215420189, 0.5795132010062691)

In [10]:
utils.save_sklearn_model(nuc_rf_model, os.path.join(MODEL_DIR, "rf_nuc_only"))
utils.save_sklearn_model(cyto_rf_model, os.path.join(MODEL_DIR, "rf_cyto_only"))

'/storage/wukevin/projects/rnagps/models/rf_cyto_only.0.21.3.skmodel'